In [ ]:
import torch.nn as nn
import torchvision
import torch.utils.data
from models import ParkingMobileNetV3, ParkingEfficientNet

In [ ]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        torchvision.transforms.Grayscale(),
        torchvision.transforms.ToTensor(),
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ParkingEfficientNet().to(device)
criterion = nn.BCEWithLogitsLoss()
model.train()

In [ ]:
# Train
num_episodes = 10

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        model.optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        model.optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/parking_efficientnet.pth")
        print("\tSaving!")
